In [1]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform
import h5py
from tqdm import tqdm_notebook,trange
%run temp.py

/home/sajid/miniconda3/envs/ipy3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
ERROR:root:File `'temp.py'` not found.


In [2]:
f = h5py.File("zp_reduced.hdf5","r")
zp_dset = f['zp_reduced']
zp_reduced = zp_dset[:,:]
suggested_step_size = zp_dset.attrs['suggested_step_size']
zp_thickness        = zp_dset.attrs['zp_thickness']
rot_slices          = zp_dset.attrs['rotation_slices']
step_xy             = zp_dset.attrs['step_xy']
expand_slices       = zp_dset.attrs['expand_slices']
step_z              = zp_dset.attrs['step_z']
max_angle           = zp_dset.attrs['max_angle']
f.close()

print("rotation_slices     :",rot_slices)
print("step_xy             :",step_xy)
print("expand_slices       :",expand_slices)
print("step_z              :",step_z)
print("zp_thickness        :",zp_thickness)

rotation_slices     : 6
step_xy             : 4.738222262408356e-09
expand_slices       : 320
step_z              : 1.5162311239706738e-06
zp_thickness        : 7.5938e-07


In [3]:
!rm zp_rotate.hdf5

In [4]:
angle = 5

In [5]:
f = h5py.File("zp_rotate.hdf5")
zp_rotate = f.create_dataset("zp_rotate",
                        (np.shape(zp_reduced)[0],np.shape(zp_reduced)[1],rot_slices),
                        chunks=True)
zp_rotate.attrs['suggested_step_size'] = suggested_step_size
zp_rotate.attrs['zp_thickness']        = 0.6845e-6
zp_rotate.attrs['step_xy']             = step_xy
zp_rotate.attrs['rotation_slices']     = rot_slices
zp_rotate.attrs['expand_slices']       = expand_slices
zp_rotate.attrs['step_z']              = step_z
zp_rotate.attrs['angle']               = angle

In [6]:
import time
t1 = time.time() 

In [7]:
zp_extent  = int((zp_thickness/step_xy)/2)
center_loc = int(expand_slices*rot_slices/2)

In [ ]:
for i in tqdm_notebook(range(15)):
    temp  = np.zeros((1000,15000,rot_slices))
    temp_ = np.zeros((1000,15000,expand_slices*rot_slices))
    transfer = np.zeros((1000,15000))
   
    #Hold no tilt pattern for initial transfer
    transfer = zp_reduced[1000*i:1000*(i+1),:]
    
    #Fill expanded array
    for j in trange(zp_extent,desc='Expand   '):    
        temp_[:,:,center_loc-j] = transfer
        temp_[:,:,center_loc+j] = transfer
    
    #Rotate expanded array
    for j in trange(1000,desc='Rotate zp'):
        temp_[j,:,:] = pyvips_rotate(temp_[j,:,:],angle_ = angle)
    
    #Collapse
    for j in trange(rot_slices,desc='Collapse '):
        temp[:,:,j] = np.sum(temp_[:,:,expand_slices*j:expand_slices*(j+1)],axis=-1)/expand_slices
    
    #Transfer back
    zp_rotate.write_direct(temp, np.s_[:,:,:], np.s_[1000*i:1000*(i+1),:,:])
    del temp_
    del temp
    del transfer

Rotate zp:  66%|██████▋   | 663/1000 [04:52<02:35,  2.17it/s]

In [ ]:
zp_rotate.attrs['angle'] = angle
f.close()

In [ ]:
t2 = time.time()
print(t2-t1)